<a href="https://colab.research.google.com/github/j28302830/Senior_Project/blob/main/reconized_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Samples and Models file

In [99]:
!wget http://dataset.tlm.unavarra.es/ransomware/samplesAndModels.tar.xz
!tar -Jxvf /content/samplesAndModels.tar.xz &> /dev/null

--2022-04-28 13:55:01--  http://dataset.tlm.unavarra.es/ransomware/samplesAndModels.tar.xz
Resolving dataset.tlm.unavarra.es (dataset.tlm.unavarra.es)... 130.206.160.87
Connecting to dataset.tlm.unavarra.es (dataset.tlm.unavarra.es)|130.206.160.87|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41519408 (40M) [application/x-xz]
Saving to: ‘samplesAndModels.tar.xz.6’

samplesAndModels.ta 100%[===================>]  39.60M  9.75MB/s    in 4.8s    

2022-04-28 13:55:06 (8.29 MB/s) - ‘samplesAndModels.tar.xz.6’ saved [41519408/41519408]



# Import Package

In [100]:
import tensorflow as tf
import keras
import glob
import numpy as np
from pickle import load
import joblib 

# Predefined Function

In [101]:
def load_sample(local_path):
  filenames = glob.glob(local_path + "/*.txt")
  for i,filename in enumerate(filenames):
    if i == 0:
      x = np.loadtxt(filename, delimiter=',', dtype = 'float32')
    else: 
      ds = np.loadtxt(filename, delimiter=',', dtype = 'float32')
      x = np.concatenate((x, ds), axis=0)

  return x

In [102]:
def accuracy(model , x, y):
  predict = model.predict(x)
  predict[predict >= 0.99] = 1
  predict[predict < 0.99] = 0

  true_values = y
  predictions = predict

  TP = ((predictions == 1) & (true_values == 1)).sum()
  FP = ((predictions == 1) & (true_values == 0)).sum()
  TN = ((predictions == 0) & (true_values == 0)).sum()
  FN = ((predictions == 0) & (true_values == 1)).sum()
  accuracy = (TP+TN) / (TP+FP+TN+FN)
  evaluate = model.evaluate(x=x, y=y)
  return [evaluate[1], accuracy]

# Load NN, CNN, and LSTM model

In [93]:
model_NN = tf.keras.models.load_model('/content/NN_CNN_LSTM_Comparison/NN')
model_CNN = tf.keras.models.load_model('/content/NN_CNN_LSTM_Comparison/CNN')
model_LSTM = tf.keras.models.load_model('/content/NN_CNN_LSTM_Comparison/LSTM')

JSON

In [94]:
json_file = open('/content/NN.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)

# Load test and train sample

In [95]:
x_test = load_sample(local_path = r'/content/samples/ransomwareSamples_test')
x_train = load_sample(local_path = r'/content/samples/ransomwareSamples_train')

# Preprocessing Sample

In [96]:
scaler = joblib.load('/content/scaler.scaler')

index = list(range(0, 30))

label = np.delete(x_test, index + [31], axis=1)
feature = np.delete(x_test, [30,31], axis=1)
feature_10X3 = np.reshape(feature, (feature.shape[0], 10, 3))



feature_scaled = scaler.transform(feature)
feature_scaled_10X3 = np.reshape(feature_scaled, (feature_scaled.shape[0], 10, 3))

print('資料集 x_test 的平均值 : ', x_test.mean(axis=0))
print('資料集 x_test 的標準差 : ', x_test.std(axis=0))
print('\nStandardScaler 縮放過後資料集 x_test 的平均值 : ', feature_scaled.mean(axis=0))
print('StandardScaler 縮放過後資料集 x_test 的標準差 : ', feature_scaled.std(axis=0))

print("feature_10X3's shape" ,feature_10X3.shape)
print("feature's shape" , x_test.shape)
print("feature's dtype" , x_test.dtype)
print("label's shape" , label.shape)
print("label's dtype" , label.dtype)



資料集 x_test 的平均值 :  [5.9634650e+05 5.9646450e+05 6.1626219e+05 6.0082194e+05 6.1096806e+05
 6.0031862e+05 5.9714181e+05 6.0455369e+05 6.1581250e+05 5.9984931e+05
 6.6018625e+05 6.5127812e+05 6.4531444e+05 6.5634431e+05 6.5318562e+05
 6.5138962e+05 6.4804638e+05 6.4783319e+05 6.4951050e+05 6.4643519e+05
 2.1893274e+01 2.1662115e+01 2.1797686e+01 2.1869705e+01 2.1969095e+01
 2.1747356e+01 2.1959156e+01 2.1660055e+01 2.1675020e+01 2.1535330e+01
 1.0000000e+00 7.5295445e+06]
資料集 x_test 的標準差 :  [1.9809578e+06 1.9829548e+06 2.0642916e+06 2.0071552e+06 2.0257402e+06
 2.0563311e+06 1.9994486e+06 2.0354585e+06 2.0783109e+06 2.0108499e+06
 2.0343956e+06 1.9871325e+06 1.9990465e+06 1.9986080e+06 1.9949164e+06
 2.0059031e+06 1.9837660e+06 1.9673376e+06 1.9897425e+06 1.9853988e+06
 8.1434433e+01 8.0579559e+01 8.1688385e+01 8.0962242e+01 8.1808128e+01
 8.1566849e+01 8.1763870e+01 8.0222404e+01 7.9913879e+01 7.9761314e+01
 0.0000000e+00 1.0596360e+08]

StandardScaler 縮放過後資料集 x_test 的平均值 :  [1.0534327 

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


# Run Model all with ransomware file

### Original Feature

In [97]:
NN = accuracy(model_NN, x=feature, y=label)
CNN = accuracy(model_CNN, x=feature_10X3, y=label)
LSTM = accuracy(model_LSTM, x=feature_10X3, y=label)
print("NN:\nevalute(): %f vs accuracy(): %f" %(NN[0], NN[1]) )
print("CNN:\nevalute(): %f vs accuracy(): %f" %(CNN[0], CNN[1]) )
print("LSTM:\nevalute(): %f vs accuracy(): %f" %(LSTM[0], LSTM[1]) )

1381/1381 [==============================] - 6s 4ms/step - loss: 8.3336 - accuracy: 0.3133
NN:
evalute(): 0.255587 vs accuracy(): 0.240916
CNN:
evalute(): 0.283276 vs accuracy(): 0.240123
LSTM:
evalute(): 0.313342 vs accuracy(): 0.215898


### Scaled feature 


In [98]:
NN = accuracy(model_NN, x=feature_scaled, y=label)
CNN = accuracy(model_CNN, x=feature_scaled_10X3, y=label)
LSTM = accuracy(model_LSTM, x=feature_scaled_10X3, y=label)
print("NN:\nevalute(): %f vs accuracy(): %f" %(NN[0], NN[1]) )
print("CNN:\nevalute(): %f vs accuracy(): %f" %(CNN[0], CNN[1]) )
print("LSTM:\nevalute(): %f vs accuracy(): %f" %(LSTM[0], LSTM[1]) )

1381/1381 [==============================] - 5s 4ms/step - loss: 9.3257 - accuracy: 0.2609
NN:
evalute(): 0.244425 vs accuracy(): 0.229301
CNN:
evalute(): 0.242410 vs accuracy(): 0.219860
LSTM:
evalute(): 0.260907 vs accuracy(): 0.229980


# Run Model with each ransomware file



---



---



# Test Scaler

In [105]:
from sklearn.preprocessing import StandardScaler
index = np.array( [ list( range(0, 30) )]  )
scaler1 = StandardScaler()
print(scaler.transform(index))
print(scaler1.transform(index))

[[-0.05201372 -0.05361435 -0.05238718 -0.05263657 -0.05211621 -0.05342558
  -0.05487471 -0.05368822 -0.05291273 -0.05325989 -0.0582454  -0.05800431
  -0.0589161  -0.05958257 -0.0593168  -0.05840374 -0.05838046 -0.05849376
  -0.05911215 -0.05746198  1.20832544  1.29770153  1.38375147  1.44517749
   1.52600189  1.60961178  1.6744698   1.74458008  1.79261236  1.8668522 ]]


NotFittedError: ignored